In [181]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import monotonically_increasing_id, format_number
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import LinearRegression

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import FMRegressor
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
import numpy as np

In [182]:
# Создание SparkSession
spark = SparkSession.builder.appName("CrabAgePrediction").getOrCreate()
# Загружаем датасет
data = spark.read.csv("CrabAgePrediction.csv", header = True, inferSchema = True)

data.printSchema()
data.show()

root
 |-- Sex: string (nullable = true)
 |-- Length: double (nullable = true)
 |-- Diameter: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Shucked Weight: double (nullable = true)
 |-- Viscera Weight: double (nullable = true)
 |-- Shell Weight: double (nullable = true)
 |-- Age: integer (nullable = true)

+---+------+--------+------+-----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|     Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+-----------+--------------+--------------+------------+---+
|  F|1.4375|   1.175|0.4125| 24.6357155|    12.3320325|     5.5848515|    6.747181|  9|
|  M|0.8875|    0.65|0.2125| 5.40057975|     2.2963095|    1.37495075|   1.5592225|  6|
|  I|1.0375|   0.775|  0.25| 7.95203475|      3.231843|    1.60174675|  2.76407625|  6|
|  F| 1.175|  0.8875|  0.25|13.48018725|    4.74854125|    2.28213475|   5.2446575| 10|
|  I|0.8875|  0.6

In [183]:
# Преобразуем Пол краба в числа
indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
# Преобразовываем данные. Начинаем с выбора признаков
features = ["SexIndex", "Length", "Diameter" , "Height", "Weight", "Shucked Weight", "Viscera Weight", "Shell Weight"]
# Создаем вектор этих признаков
assembler = VectorAssembler(inputCols=features, outputCol="features")
# Разделяем данные на обучение и тест
(train, test) = data.randomSplit([0.7, 0.3])

# Объявляем необходимые переменные
feature_names = test.columns
feature_names.remove("Age")

tupleRMSE = ()
Pred_arr = []
Importance_arr = []

print("Train count: " + str(train.count()))
train.show(3)

print("Test count: " + str(test.count()))
test.show(3)

Train count: 2739
+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F| 0.725|   0.525|0.1875| 7.7961125|     3.2034935|    1.91359125|   0.9922325|  6|
|  F| 0.725|  0.5625|0.1875|   3.96893|    1.45999925|    0.66621325|     1.13398|  5|
|  F|0.7625|  0.5625| 0.175|4.20990075|    1.65844575|    0.94970825|   1.2757275|  7|
+---+------+--------+------+----------+--------------+--------------+------------+---+
only showing top 3 rows

Test count: 1154
+---+------+--------+------+----------+--------------+--------------+------------+---+
|Sex|Length|Diameter|Height|    Weight|Shucked Weight|Viscera Weight|Shell Weight|Age|
+---+------+--------+------+----------+--------------+--------------+------------+---+
|  F|0.6875|  0.4875| 0.175|   2.26796|     0.8788345|    0.60951425| 

In [184]:
# Создаем модель
rf = RandomForestRegressor(featuresCol = "features", labelCol= "Age")
# Создаем конвейер
pipelineRF = Pipeline(stages = [indexer, assembler, rf])
# Обучение модели
modelRF = pipelineRF.fit(train)
# Предсказание возраста на тестовой выборке
predictionsRF = modelRF.transform(test).withColumnRenamed("prediction", "predictionsRF")
Pred_arr.append(predictionsRF)

# Сохранение степени влиятельности признаков
name = "RandomForestRegressor"
tup = (name, ) + tuple(list(map(lambda x: float(x), modelRF.stages[2].featureImportances)))
Importance_arr.append(tup)
print(Importance_arr)

[('RandomForestRegressor', 0.027119910784811845, 0.08329306569979276, 0.07078979998147814, 0.15699721607736375, 0.15581790686810287, 0.0593149986253117, 0.04934195715846649, 0.39732514480467246)]


In [185]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsRF", metricName = "rmse")
rmseRF = evaluator.evaluate(predictionsRF)
tupleRMSE += (rmseRF,)
print("Root Mean Squared Error (RMSE): %s" % rmseRF)
# Вывод результата и выключение SparkSession
predictionsRF.select("Age", "predictionsRF").show()

Root Mean Squared Error (RMSE): 2.2617380446755893
+---+------------------+
|Age|     predictionsRF|
+---+------------------+
|  5|5.3762516618745835|
|  7|  6.57216564001362|
|  7| 7.751587140618805|
|  9|  7.87059131440068|
|  8| 8.079647366910505|
| 10| 8.800286438150124|
| 10|  8.65372018357122|
|  7| 8.858643134634793|
| 11| 8.907275261325367|
| 10| 9.034079031576242|
| 10| 9.685681992834322|
|  8| 9.541083574794659|
|  7| 9.331843450571363|
|  7| 9.410622345746514|
|  9| 9.431524938292133|
| 11| 9.354394511290602|
| 11|  9.58661408026778|
|  9| 9.334067963163523|
|  9| 8.952319693574477|
| 12| 8.976752072780265|
+---+------------------+
only showing top 20 rows



In [186]:
# Создание модели градиентного бустинга
gbt = GBTRegressor(featuresCol="features", labelCol= "Age")
# Создание конвейера
pipelineGBT = Pipeline(stages = [indexer, assembler, gbt])
# Обучение модели
modelGBT = pipelineGBT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGBT = modelGBT.transform(test).withColumnRenamed("prediction", "predictionsGBT")
Pred_arr.append(predictionsGBT)

# Сохранение степени влиятельности признаков
name = "GBTRegressor"
tup = (name, ) + tuple(list(map(lambda x: float(x), modelGBT.stages[2].featureImportances)))
Importance_arr.append(tup)
print(Importance_arr)

[('RandomForestRegressor', 0.027119910784811845, 0.08329306569979276, 0.07078979998147814, 0.15699721607736375, 0.15581790686810287, 0.0593149986253117, 0.04934195715846649, 0.39732514480467246), ('GBTRegressor', 0.03844092535145324, 0.08420753666699873, 0.0874690012714675, 0.09498669333940496, 0.12022822146921452, 0.17856449911902147, 0.06304816219232512, 0.3330549605901143)]


In [187]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGBT", metricName = "rmse")
rmseGBT = evaluator.evaluate(predictionsGBT)
tupleRMSE += (rmseGBT,)
print("Root Mean Squared Error (RMSE): %s" % rmseGBT)
# Вывод результата и выключение SparkSession
predictionsGBT.select("Age", "predictionsGBT").show()

Root Mean Squared Error (RMSE): 2.297819877047723
+---+------------------+
|Age|    predictionsGBT|
+---+------------------+
|  5|5.5596744527167825|
|  7| 6.022912461137717|
|  7| 9.078273622627519|
|  9| 9.371252994015094|
|  8| 8.957174704980082|
| 10| 9.124165569735737|
| 10| 9.146131552874726|
|  7| 6.781412787206129|
| 11| 8.989029562898878|
| 10|  9.03371312734529|
| 10|10.734413408110344|
|  8| 9.779648295660518|
|  7| 9.419386374219382|
|  7|  9.12331393501033|
|  9|10.750482352650199|
| 11| 9.419386374219382|
| 11|10.806102904806979|
|  9| 8.823128895711541|
|  9| 8.952028406968632|
| 12|  9.12331393501033|
+---+------------------+
only showing top 20 rows



In [188]:
# Создание модели линейной регрессии
lr = LinearRegression(featuresCol="features", labelCol="Age", maxIter=10, regParam=0.3, elasticNetParam=0.8) 
# Создание конвейера
pipelineLR = Pipeline(stages = [indexer, assembler, lr])
# Fit the model
modelLR = pipelineLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsLR = modelLR.transform(test).withColumnRenamed("prediction", "predictionsLR")
Pred_arr.append(predictionsLR)

In [189]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsLR", metricName = "rmse")
rmseLR = evaluator.evaluate(predictionsLR)
tupleRMSE += (rmseLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseLR)
# Вывод результата и выключение SparkSession
predictionsLR.select("Age", "predictionsLR").show()

Root Mean Squared Error (RMSE): 2.441501947834654
+---+-----------------+
|Age|    predictionsLR|
+---+-----------------+
|  5| 7.29276625844087|
|  7|7.603766781483595|
|  7|7.822837038897525|
|  9|7.846368333406389|
|  8|8.065440914967235|
| 10|8.243173199089549|
| 10|8.155664791058577|
|  7| 9.00235075848028|
| 11|8.385287477352524|
| 10|8.304226056405955|
| 10|8.678192290300956|
|  8|8.737224666424618|
|  7|8.822418913657469|
|  7|8.893622892591646|
|  9|8.748295326811427|
| 11|8.718922492331908|
| 11|8.919988903552337|
|  9|8.593403998624186|
|  9|8.490600240019331|
| 12|8.795763871385237|
+---+-----------------+
only showing top 20 rows



In [190]:
# Создание модели решающих деервьев
dt = DecisionTreeRegressor(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineDT = Pipeline(stages = [indexer, assembler, dt])
# Fit the model
modelDT = pipelineDT.fit(train)
# Предсказание возраста на тестовой выборке
predictionsDT = modelDT.transform(test).withColumnRenamed("prediction", "predictionsDT")
Pred_arr.append(predictionsDT)

# Сохранение степени влиятельности признаков
name = "DecisionTreeRegressor"
tup = (name, ) + tuple(list(map(lambda x: float(x), modelDT.stages[2].featureImportances)))
Importance_arr.append(tup)
print(Importance_arr)

[('RandomForestRegressor', 0.027119910784811845, 0.08329306569979276, 0.07078979998147814, 0.15699721607736375, 0.15581790686810287, 0.0593149986253117, 0.04934195715846649, 0.39732514480467246), ('GBTRegressor', 0.03844092535145324, 0.08420753666699873, 0.0874690012714675, 0.09498669333940496, 0.12022822146921452, 0.17856449911902147, 0.06304816219232512, 0.3330549605901143), ('DecisionTreeRegressor', 0.03415995798991525, 0.00221460423545631, 0.007546978713055943, 0.009024617075236913, 0.0, 0.1385979769941694, 6.739740738945022e-05, 0.8083884675847767)]


In [191]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsDT", metricName = "rmse")
rmseDT = evaluator.evaluate(predictionsDT)
tupleRMSE += (rmseDT,)
print("Root Mean Squared Error (RMSE): %s" % rmseDT)
# Вывод результата и выключение SparkSession
predictionsDT.select("Age", "predictionsDT").show()

Root Mean Squared Error (RMSE): 2.3152853226399177
+---+------------------+
|Age|     predictionsDT|
+---+------------------+
|  5|               5.5|
|  7| 5.884057971014493|
|  7| 9.087719298245615|
|  9| 9.087719298245615|
|  8| 9.087719298245615|
| 10| 9.087719298245615|
| 10| 9.087719298245615|
|  7|              7.96|
| 11| 9.087719298245615|
| 10| 9.087719298245615|
| 10|10.402985074626866|
|  8| 9.087719298245615|
|  7| 9.087719298245615|
|  7| 9.087719298245615|
|  9|10.402985074626866|
| 11| 9.087719298245615|
| 11|10.402985074626866|
|  9|10.402985074626866|
|  9| 9.087719298245615|
| 12| 9.087719298245615|
+---+------------------+
only showing top 20 rows



In [192]:
# Создание модели изотонической регрессии
ir = IsotonicRegression(featuresCol="features", labelCol="Age")
# Создание конвейера
pipelineIR = Pipeline(stages = [indexer, assembler, ir])
# Fit the model
modelIR = pipelineIR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsIR = modelIR.transform(test).withColumnRenamed("prediction", "predictionsIR")
Pred_arr.append(predictionsIR)

In [193]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsIR", metricName = "rmse")
rmseIR = evaluator.evaluate(predictionsIR)
tupleRMSE += (rmseIR,)
print("Root Mean Squared Error (RMSE): %s" % rmseIR)
# Вывод результата и выключение SparkSession
predictionsIR.select("Age", "predictionsIR").show()

Root Mean Squared Error (RMSE): 3.8313767338800853
+---+------------------+
|Age|     predictionsIR|
+---+------------------+
|  5|10.177153920619554|
|  7|10.177153920619554|
|  7|10.177153920619554|
|  9|10.177153920619554|
|  8|10.177153920619554|
| 10|10.177153920619554|
| 10|10.177153920619554|
|  7|10.177153920619554|
| 11|10.177153920619554|
| 10|10.177153920619554|
| 10|10.177153920619554|
|  8|10.177153920619554|
|  7|10.177153920619554|
|  7|10.177153920619554|
|  9|10.177153920619554|
| 11|10.177153920619554|
| 11|10.177153920619554|
|  9|10.177153920619554|
|  9|10.177153920619554|
| 12|10.177153920619554|
+---+------------------+
only showing top 20 rows



In [194]:
# Создание модели FMRegressor
fm = FMRegressor(featuresCol="features", labelCol="Age", predictionCol="prediction",stepSize=0.001)
# Создание конвейера
pipelineFM = Pipeline(stages = [indexer, assembler, fm])
# Fit the model
modelFM = pipelineFM.fit(train)
# Предсказание возраста на тестовой выборке
predictionsFM = modelFM.transform(test).withColumnRenamed("prediction", "predictionsFM")
Pred_arr.append(predictionsFM)

In [195]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsFM", metricName = "rmse")
rmseFM = evaluator.evaluate(predictionsFM)
tupleRMSE += (rmseFM,)
print("Root Mean Squared Error (RMSE): %s" % rmseFM)
# Вывод результата и выключение SparkSession
predictionsFM.select("Age", "predictionsFM").show()

Root Mean Squared Error (RMSE): 5.102875633263086
+---+------------------+
|Age|     predictionsFM|
+---+------------------+
|  5|0.6684091401699361|
|  7|1.0686951841741683|
|  7|1.2842983531497907|
|  9| 1.320192774126975|
|  8|1.4606669908620329|
| 10|1.7823101604691705|
| 10|1.6913423062896555|
|  7|4.3647012296127015|
| 11| 2.381788143704353|
| 10| 2.079483997915977|
| 10|2.5115753001418737|
|  8|2.7104887058823754|
|  7|2.7340487377828873|
|  7| 3.003722709507238|
|  9|2.6749109187007267|
| 11|2.7861029359282217|
| 11| 3.127531259045429|
|  9|2.9305381354260325|
|  9| 2.804319857024282|
| 12|3.2994431689393093|
+---+------------------+
only showing top 20 rows



In [196]:
# Модель  `AFTSurvivalRegression`  не подходит для датасета  `CrabAgePrediction`, так как нет информации о том,  
# живы ли были крабы, когда их поймали.

# Эта модель предназначена для анализа данных выживания, где есть данные о времени до события (в данном случае, смерть) 
# и информация о том,  было ли это событие наблюдено (краб умер) или  **цензурировано**  (краб был жив, когда его поймали).  

In [197]:
glr = GeneralizedLinearRegression(featuresCol="features", labelCol="Age", family="gaussian", link="identity", maxIter=10, regParam=0.3)
# Создание конвейера
pipelineGLR = Pipeline(stages = [indexer, assembler, glr])
# Fit the model
modelGLR = pipelineGLR.fit(train)
# Предсказание возраста на тестовой выборке
predictionsGLR = modelGLR.transform(test).withColumnRenamed("prediction", "predictionsGLR")
Pred_arr.append(predictionsGLR)

In [198]:
# Оценка модели
evaluator = RegressionEvaluator(labelCol = "Age", predictionCol = "predictionsGLR", metricName = "rmse")
rmseGLR = evaluator.evaluate(predictionsGLR)
tupleRMSE += (rmseGLR,)
print("Root Mean Squared Error (RMSE): %s" % rmseGLR)
# Вывод результата и выключение SparkSession
predictionsGLR.select("Age", "predictionsGLR").show()

Root Mean Squared Error (RMSE): 2.243629380674696
+---+-----------------+
|Age|   predictionsGLR|
+---+-----------------+
|  5|6.654106716435999|
|  7|7.110659191324778|
|  7|7.520945693707615|
|  9|7.653423681610279|
|  8|8.028030558921165|
| 10|8.358074962723453|
| 10|8.200605567695213|
|  7| 8.08108056898484|
| 11|8.171090535879504|
| 10|8.446610511580783|
| 10|8.974842535705207|
|  8|8.886981722080945|
|  7| 9.09171617368003|
|  7|8.953629963939004|
|  9|9.028107273651347|
| 11|9.046218912801681|
| 11|9.167057997508405|
|  9|8.685971737316043|
|  9| 8.48363454055028|
| 12|8.922531106070661|
+---+-----------------+
only showing top 20 rows



In [199]:
# Создаем списки моделей, выполняющихся том же в порядке выше
modelsList = ["RandomForestRegressor", "GBTRegressor", "LinearRegression", "DecisionTreeRegressor", 
              "IsotonicRegression", "FMRegressor", "GeneralizedLinearRegression"]
shortModelsList = ["predictionsRF", "predictionsGBT", "predictionsLR", "predictionsDT", "predictionsIR", 
                   "predictionsFM", "predictionsGLR"]

# Схема нового DataFrame значений RMSE каждой модели
schemaRMSE = StructType([
    StructField(modelsList[0], DoubleType(), True),
    StructField(modelsList[1], DoubleType(), True),
    StructField(modelsList[2], DoubleType(), True),
    StructField(modelsList[3], DoubleType(), True),
    StructField(modelsList[4], DoubleType(), True),
    StructField(modelsList[5], DoubleType(), True),
    StructField(modelsList[6], DoubleType(), True)
])
# Создание DataFrame
rmseDF = spark.createDataFrame([tupleRMSE], schemaRMSE)

In [200]:
types = [StructField("Model_name", StringType(), True)]

for i in feature_names:
    types.append(StructField(i, DoubleType(), True))

schema_imp = StructType(types)
impDF = spark.createDataFrame(Importance_arr, schema_imp)
impDF.printSchema()

root
 |-- Model_name: string (nullable = true)
 |-- Sex: double (nullable = true)
 |-- Length: double (nullable = true)
 |-- Diameter: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Shucked Weight: double (nullable = true)
 |-- Viscera Weight: double (nullable = true)
 |-- Shell Weight: double (nullable = true)



In [201]:
for i, df in enumerate(Pred_arr):
    df = df.withColumn("id", monotonically_increasing_id())
    Pred_arr[i] = df

ListSelectedDF = [df.select("Age", "id", shortModelsList[i]) for i, df in enumerate(Pred_arr)]
predDF = ListSelectedDF[0]

for i in range(1, len(ListSelectedDF)):
    predDF = predDF.join(ListSelectedDF[i], on = ["Age", "id"], how = "inner")

predDF = predDF.drop("id")

In [202]:
# Вывод лучшего результата
print(f"Лучшая модель: {modelsList[rmseDF.head().index(min(rmseDF.head()))]} (RMSE: {min(rmseDF.head())})")
# Запись результатов в json-файл
rmseDF.coalesce(1).write.json("RMSE.json", mode = 'overwrite')
predDF.write.json("Pred.json", mode = 'overwrite')
impDF.coalesce(1).write.json("Importance.json", mode = 'overwrite')

Лучшая модель: GeneralizedLinearRegression (RMSE: 2.243629380674696)


In [203]:
spark.stop()